# Configure non-linear integer optimization problem in Genetic Algorithm

Genetic Algorithm is one of oldest heuristic algorithm designed to find solution in the space of Operations Research. Many python programs are written on various application of GA in different famous problems. However, design a GA in case of real world problem like non-linear and constrained solution is difficult to find. In this notebook, not only we shall solve this, but we shall create kind of code base which can be tweaked for similar and more complex solution.

## Problem Statement - Modified Rosenbrok Function

maximize z = 120 * (x1 - x1*x2)^2 + 100 * (x2 - x1^2)^3

x1 + x2 <= 49 ---------- (1)
2*x1 + x2^2 <= 166 -------- (2)
x1, x2>0 -------------- (3)


In [68]:
import random
# random.seed(23)

# parameter initialization
max_val_x1 = 48 # maximum value of x1 possible
max_val_x2 = 48 # maximum value of x2 possible
pop_size = 50 # population size; preferred to be as maximum as possible

max_iter = 100 # maximum numebr of iteration run
max_repeat = 20  #

# variable space preparation
input_array_x1 = [i+1 for i in range(max_val_x1)] # possible list of x1 values
input_array_x2 = [i+1 for i in range(max_val_x2)] # possible list of x2 values
max_length_x1 = len(bin(input_array_x1[-1]))-2 # After binary conversion, what is the maximum length of x1
max_length_x2 = len(bin(input_array_x2[-1]))-2 # After binary conversion, what is the maximum length of x2

In [69]:
# Auxillary Functions - set I

def convert_to_binary(number, length):
  '''
  Converts a number to binary with defined length with pre-filled zeros at the beginning
  '''
  return format(number, f'0{length}b')

def create_chromosome(input_array, length):
  '''
  Create chromosomes from a randomly selected value from input array
  '''
  val = random.choice(input_array)
  return(convert_to_binary(val, length))

def create_initial_population(input_array_x1, input_array_x2, pop_size, max_length_x1, max_length_x2):
  '''
  Create initial population of chromosomes
  '''
  population = []
  while len(population) <pop_size:
      chromosome1 = create_chromosome(input_array_x1, max_length_x1)
      chromosome2 = create_chromosome(input_array_x2, max_length_x2)
      # check pair validity
      validity = chromosome_validation(chromosome1, chromosome2)
      if (validity == 1):
          population.append([chromosome1, chromosome2])
  return population

In [70]:
# Auxillary Functions - set II

def constraint_validation1(chromosome1, chromosome2):
  '''
  Check if the constraint number 1 is violated or not
  '''
  value = int(chromosome1,2) + int(chromosome2,2)
  if value>49:
      return 0
  else:
      return 1

def constraint_validation2(chromosome1, chromosome2):
  '''
  Check if the constraint number 2 is violated or not
  '''
  value = int(chromosome1,2)*2 + int(chromosome2,2)**2
  if value>166:
      return 0
  else:
      return 1

def chromosome_validation(chromosome1, chromosome2):
  '''
  Check overall if a chromosome validates all the constraints or not
  '''
  status1 = constraint_validation1(chromosome1, chromosome2)
  status2 = constraint_validation2(chromosome1, chromosome2)
  return(status1*status2)

def cross_breeding(chromosome1, chromosome2):
  '''
  Cross breeding of two chromosomes
  '''
  new_chromosome = ''
  for g1, g2 in zip(chromosome1, chromosome2):
      if(random.random()<0.9):
          new_chromosome = new_chromosome + str(random.choice([g1,g2]))
      else:
          new_chromosome = new_chromosome + str(random.choice(['1','0']))
  return new_chromosome

def validated_new_chromosome(pair_1, pair_2):
  '''
  This function ensures that we have chromosomes which validate all the constraints. Until such chromosoem is obtained, the iteration will continue
  '''
  x1 = pair_1[0]
  x1_hash = pair_2[0]
  x2 = pair_1[1]
  x2_hash = pair_2[1]
  is_valid = 0

  while (is_valid != 1):
    x1_new = cross_breeding(x1, x1_hash)
    x2_new = cross_breeding(x2, x2_hash)
    is_valid = chromosome_validation(x1_new, x2_new)

  return ([x1_new, x2_new])

In [71]:
# Auxillary Function III
def create_new_generation(sorted_population, pop_size):
  '''
  Create new generation from the sorted population of targeted population size
  '''
  new_generation = []

  # 10% goes to the next generation
  s_10 = int(10*pop_size/100)
  new_generation.extend(sorted_population[:s_10])

  # 60% of top chromosomes will cross-breed and mutate among themselves
  s_60 = int(60*pop_size/100)
  mating_pool = sorted_population[:s_60]

  while len(new_generation) < pop_size:
      mutated_pair = validated_new_chromosome(random.choice(mating_pool),
                                                    random.choice(mating_pool))
      new_generation.append(mutated_pair)
  return new_generation

def objective_value(pair):
  '''
  Objective function to be maximized
  '''
  x1=int(pair[0],2)
  x2=int(pair[1],2)
  value = 120*(x1 - x1*x2)**2 + 100*(x2 - x1**2)**3
  return value

def stopping_condition(previous_val, current_val, current_iter, max_iter, value_count, max_repeat):
  '''
  condition 1 : iteration numbers reached
  condition 2 : no improvements for last n iterations
  there can also be a local dip. Hence, we need to store best generation
  after every iteration
  '''
  if(current_val>previous_val):
      previous_val = current_val
      value_count = 0
  if(current_val == previous_val):
      value_count+=1

  if((current_iter>=max_iter) or (value_count>=max_repeat)):
      return 0, value_count, previous_val
  else:
      return 1, value_count, previous_val

In [72]:
###------ Actual Run Function ------###

# initial population generation
init_pop = create_initial_population(input_array_x1, input_array_x2, pop_size, max_length_x1, max_length_x2)

# initialize runtime parameters
current_pop = init_pop
current_iter = 0
previous_val = 0
current_val = 0
value_count = 0

# First run output
sorted_pop = sorted(current_pop, key=lambda x : objective_value(x), reverse=1)
best_pair = sorted_pop[0]
current_val = objective_value(best_pair)

# validate stopping condition
cont_iter, value_count, previous_val = stopping_condition(previous_val, current_val, current_iter, max_iter, value_count, max_repeat)

In [73]:
# Run remaining iterations

while (cont_iter>0):

    new_gen = create_new_generation(sorted_pop, pop_size)
    sorted_pop = sorted(new_gen, key=lambda x : objective_value(x), reverse=1)
    best_pair = sorted_pop[0]
    current_val = objective_value(best_pair)
    print("Current Iteration : ", current_iter+1)
    print("Current Best Value : ", current_val)
    cont_iter, value_count, previous_val = stopping_condition(previous_val, current_val, current_iter, max_iter, value_count, max_repeat)
    current_iter+=1

print("Optimum Value : ", previous_val)
print("X1 : ", int(best_pair[0],2))
print("X2 : ", int(best_pair[1],2))

Current Iteration :  1
Current Best Value :  133920
Current Iteration :  2
Current Best Value :  133920
Current Iteration :  3
Current Best Value :  133920
Current Iteration :  4
Current Best Value :  133920
Current Iteration :  5
Current Best Value :  172800
Current Iteration :  6
Current Best Value :  179500
Current Iteration :  7
Current Best Value :  225920
Current Iteration :  8
Current Best Value :  225920
Current Iteration :  9
Current Best Value :  225920
Current Iteration :  10
Current Best Value :  225920
Current Iteration :  11
Current Best Value :  225920
Current Iteration :  12
Current Best Value :  225920
Current Iteration :  13
Current Best Value :  225920
Current Iteration :  14
Current Best Value :  225920
Current Iteration :  15
Current Best Value :  225920
Current Iteration :  16
Current Best Value :  225920
Current Iteration :  17
Current Best Value :  225920
Current Iteration :  18
Current Best Value :  225920
Current Iteration :  19
Current Best Value :  225920
Cu